<a href="https://colab.research.google.com/github/cobyoram/python-for-data-scientists/blob/master/BigData_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weather's Effect on Solar Generation

## 1. Introduction
As a solar salesman and business owner, I have domain experience in the renewable energy space. While solar may seem like the best solution to energy production. However, solar can only produce so much power during so many hours of the day. The goal of this model is to try to predict solar production using the production readings of other generation methods and weather data.

## 2. Question

### Can I predict solar generation using weather data and other generation readings for the day?

## 3. Data
The data comes from [kaggle](https://www.kaggle.com/nicholasjhana/energy-consumption-generation-prices-and-weather) split into two datasets. The first contains 35,064 observations. These observations include a column of instantaeously captured production levels of solar systems for each hour for four years, aggregated across Spain. The second database includes weather information at the same resolution for each of the five major cities in Spain. I intend to average the cities' weather data to represent the effective weather conditions for all of spain.

In [0]:
!pip install dask_ml --quiet
!pip install -U ipykernel --quiet

import dask.dataframe as dd
from dask_ml.model_selection import train_test_split

     |████████████████████████████████| 133kB 9.1MB/s 
     |████████████████████████████████| 634kB 44.6MB/s 
     |████████████████████████████████| 501kB 54.9MB/s 
     |████████████████████████████████| 122kB 8.8MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.2.1 which is incompatible.


ContextualVersionConflict: ignored

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns


In [0]:
try:
    # For local environment, load in csvs from directory
    raw_energy = dd.read_csv('energy_dataset.csv')
    raw_weather = dd.read_csv('weather_features.csv')
except FileNotFoundError as e:
    # Load the dataframes from their location online if that doesn't work
    print('Attempting to read datasets from online csv files on Github')
    raw_energy = dd.read_csv('https://raw.githubusercontent.com/cobyoram/Experimental_Design__Thinkful_capstone_1/master/energy_dataset.csv')
    raw_weather = dd.read_csv('https://raw.githubusercontent.com/cobyoram/Experimental_Design__Thinkful_capstone_1/master/weather_features.csv')

In [0]:
# Familiarize with energy data
print(raw_energy.info())
raw_energy.head()

In [0]:
# Familiarize with weather data
print(raw_weather.info())
raw_weather.head()

## Clean the data

In [0]:
# We want to be able to combine these datasets together. So we need to aggregate the weather data across spain, and set a shared key variable using the date and time
# First remove the +01:00 and like-formatted timezone aware attributes from datetime string in each dataframe
raw_weather['dt_iso'] = raw_weather['dt_iso'].str.replace('\+[0-9][0-9]:[0-9][0-9]', '', regex=True)
raw_energy['time'] = raw_energy['time'].str.replace('\+[0-9][0-9]:[0-9][0-9]', '', regex=True)

In [0]:
raw_weather.head()

In [0]:
# Define a new meta to map partitions to: datetime object type
meta = ('dt_iso', 'datetime64[ns]')

# Map the values to the new datetime objects
raw_weather['date'] = raw_weather['dt_iso'].map_partitions(pd.to_datetime, meta=meta)

# Drop old datetime string columns
raw_weather = raw_weather.drop('dt_iso', axis=1)
raw_weather.head()

In [0]:
# Ensure the transformation worked
type(raw_weather)

In [0]:
# Repeat for solar dataset
meta = ('time', 'datetime64[ns]')
raw_energy['date'] = raw_energy['time'].map_partitions(pd.to_datetime, meta=meta)
raw_energy = raw_energy.drop('time', axis=1)
raw_energy.head()

In [0]:
raw_energy.date.dtype

In [0]:
# We group the weather data by the date, aggregating by mean for numerical data, and mode for categorical data
agg_weather_num = raw_weather[['date'] + list(raw_weather.select_dtypes('number').columns)].groupby(by='date').mean()
agg_weather_cat = raw_weather[['date'] + list(raw_weather.select_dtypes('object').columns)].groupby(by='date').max()
# We concat the two aggregated weather datasets together to form one
agg_weather = agg_weather_cat.join(agg_weather_num, on='date').reset_index()
agg_weather.head()

In [0]:
raw_energy.date.isna().sum().compute()

In [0]:
agg_weather['date'].count().compute()

In [0]:
raw_energy['date'].count().compute()

In [0]:
first_date = agg_weather.loc[0, 'date'].compute()[0]

# agg_weather['joincol'] = 

In [0]:
first_date

In [0]:
first_date_e = raw_energy.loc[0, 'date'].compute()[0]
first_date_e

In [0]:
# Now we join the weather and energy datasets together on the date column

energy_weather_data = agg_weather.merge(raw_energy, on='date')


In [0]:
energy_weather_data.head()

In [0]:
def missingness_summary(df, **kwargs):
    '''
    This function creates a series representing what percentage of data is null for each column of a dataframe

    You can use a number of kwargs to customize the function. Those include:
    kwargs:
        print_log   -   [True, False]: If true, will print the output before returning the Series (default False)
        sort        -   ['asc', 'desc']: Allows you to sort the data by ascending or descending (default 'desc')
    
    Returns Series with index = column names and value = percentage of nulls in column

    '''
    s = df.isna().sum()*100/len(df)

    sort = 'desc'
    if kwargs.get('sort'):
        sort = kwargs.get('sort')
    if sort == 'asc':
        s.sort_values(ascending=True, inplace=True)
    elif sort == 'desc':
        s.sort_values(ascending=False, inplace=True)

    print_log = False
    if kwargs.get('print_log'):
        print_log = kwargs.get('print_log')
    if print_log == True:
        print(s)

    return s

In [0]:
# Next we get rid of columns that are missing too much data
missings = energy_weather_data.isna().sum()/len(energy_weather_data)

In [0]:
missings.compute()

In [0]:
drop_cols = [col for col in missings.loc[missings == 1].index]
ew_clean = energy_weather_data.drop(drop_cols, axis=1)


In [0]:
drop_cols

In [0]:
ew_clean.head()

In [0]:
def myinterp (xs, secs=[], vals=[]):
    ret = np.interp(xs, secs, vals)
    return ret

In [0]:
# There is not much missing data now, so we'll drop the missing values to have a non-null dataset
ew_clean_nn = ew_clean.dropna()

In [0]:
def repeats_summary(df, sort='desc', print_log=False, value_agg='none', value=0):
    repeats_percents = []
    print_value = []
    for col in df.columns:
        if value_agg == 'none':
            value = value
            print_value = []
        elif value_agg == 'mode':
            value = df[col].mode().iloc[0]
        elif value_agg == 'mean':
            value = df[col].mean().iloc[0]
        elif value_agg == 'median':
            value = df[col].median().iloc[0]
        elif value_agg == 'max':
            value = df[col].max().iloc[0]
        elif value_agg == 'min':
            value = df[col].min().iloc[0]
        else: raise ValueError('Wrong entry for \'value_agg\'. Will accept \'mode\', \'mean\', \'median\', \'max\', \'min\'')

        repeats_percents.append(len(df.loc[df[col] == value])*100/len(df))
        print_value.append(value)

    S = pd.Series(repeats_percents, index=df.columns)
    if sort == 'desc':
        S = S.sort_values(ascending=False)
    elif sort == 'asc':
        S = S.sort_values(ascending=True)
    elif sort == 'none':
      None
    else: raise ValueError('Wrong entry for \'sort\'. Will accept \'asc\' or \'desc\'')

    if print_log:
        print(f'Repeated values: {print_value}\n{S}')

    return S

def drop_null_rows(df):
    for col in df.columns:
        df.drop(df.loc[df[col].isnull()].index, axis=0, inplace=True)
    return df

In [0]:
# We drop columns that repeat 0 a ridiculous amount of times over the 3 years.
repeats = repeats_summary(ew_clean_nn, sort='none', print_log=True, value=0)

In [0]:
ew_clean_nn = ew_clean_nn.drop(repeats.loc[repeats > 90].index, axis=1)

## Feature Selection

In [0]:
# Print out the unique classes in each categorical variable
for col in ew_clean_nn.select_dtypes('object').columns:
    print(col)
    print(ew_clean[col].unique())

In [0]:
# Drop categorical variables that we won't need or that have too many unique clases
cat_drop_cols = ['city_name', 'weather_description', 'weather_icon']
ew_feats = ew_clean_nn.drop(cat_drop_cols, axis=1)

In [0]:
dum_cols = ew_feats.select_dtypes('object').columns
dum_cols

In [0]:
ew_feats.categorize(ew_feats[dum_cols]).head()

In [0]:
# Create dummy variables for categorical columns

dums = dd.get_dummies(ew_feats.categorize(ew_feats[dum_cols]), drop_first=True)
dums

In [0]:
dums.head()


In [0]:
# Check for multicollinearity
corr = dums.corr()
plt.figure(figsize=(17,15))
sns.heatmap(corr)
plt.show()

In [0]:
# Drop collinear variables after interpreting the heatmap
manual_drop_cols = [col for col in dums.columns if 'forecast' in col or 'price' in col or 'total' in col]
manual_drop_cols += ['temp_min', 'temp_max']
manual_drop_cols

In [0]:
dums = dums.drop(manual_drop_cols, axis=1)
dums

In [0]:
# Explore the behavior of generation columns to help us choose a model
generation_cols = [col for col in dums.columns if 'generation' in col]
plot_df = dums[generation_cols + ['date']]
melted_df = plot_df.melt(id_vars='date')
plt.figure(figsize=(15,7))
sns.lineplot(x=melted_df['date'].dt.hour, y=melted_df['value'], hue=melted_df['variable'])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2)
plt.show()

In [0]:
import math
def make_subplots(df, plotfunc=None, func_args=[], func_kwargs={}, limitx=8, each_size=3, **kwargs):
    '''
    Makes a subplot, filled with a given plotting function
    '''
    columns = df.columns
    len_cols = len(columns)

    try_num = len_cols
    while True:
        sq = math.sqrt(try_num)
        if sq == int(sq):
            break
        try_num += 1
    count_dimensions = tuple([sq, sq])

    if count_dimensions[0] > limitx:
        count_dimensions = tuple([int(len_cols/limitx) + 1, limitx])

    dimensions = tuple([count_dimensions[1] * each_size, count_dimensions[0] * each_size])
    plt.figure(figsize=dimensions)

    for i, col in enumerate(columns, 1):
        plt.subplot(count_dimensions[0], count_dimensions[1], i)
        plotfunc(df, col, *func_args, **func_kwargs)
        plt.title(col)

    plt.tight_layout()
    plt.show()

In [0]:
# Explore the distributions of the weather variables
weather_main_cols = [col for col in dums.columns if 'weather_main' in col]
dist_df = dums.drop(generation_cols + weather_main_cols + ['date'], axis=1)

def plot_dist(df, col):
    sns.distplot(dist_df[col])

make_subplots(dist_df, plot_dist)

In [0]:
dums['hour'] = dums['date'].dt.hour
dums['month'] = dums['date'].dt.month
dums['year'] = dums['date'].dt.year

In [0]:
import joblib
from dask_ml.model_selection import train_test_split

# Spleat features and target into train and test splits
X = dums.drop(['generation solar', 'date'], axis = 1)
Y = dums['generation solar']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .1)

X_train.persist()
X_test.persist()
Y_train.persist()
Y_test.persist()

In [0]:
from sklearn.ensemble import RandomForestRegressor
from dask.distributed import Client
client = Client()

# Test the Random Forest Regressor model
rfr = RandomForestRegressor(max_depth=8)

with joblib.parallel_backend('dask'):
  rfr.fit(X_train, Y_train)

print(rfr.score(X_train, Y_train))
print(rfr.score(X_test, Y_test))

In [0]:
from sklearn.ensemble import GradientBoostingRegressor

# Test the Gradient Boosting Regressor model
gbr = GradientBoostingRegressor()

with joblib.parallel_backend('dask'):
  gbr.fit(X_train, Y_train)
  
print(gbr.score(X_train, Y_train))
print(gbr.score(X_test, Y_test))



In [0]:
pd.Series(gbr.feature_importances_, index=X_train.columns).sort_values(ascending=False)

In [0]:

from sklearn.linear_model import LinearRegression

# Test a linear regression model
lrm = LinearRegression()

with joblib.parallel_backend('dask'):
  lrm.fit(X_train, Y_train)
  
print(lrm.score(X_train, Y_train))
print(lrm.score(X_test, Y_test))

In [0]:
from dask_ml.model_selection import GridSearchCV as DaskGS

In [0]:
gs = DaskGS(
    gbr,
    param_grid = {'n_estimators': [10, 100, 500],
                        'max_features': [2,4,8],
                        # 'learning_rate': [.05,.1,.2,.4,.8,1],
                        'max_depth': [2,4,8]},
    cv=5)

gs.fit(X_train, Y_train)


print(gs.best_params_)
print(gs.best_score_)

In [0]:
# Implement the Random Forest Regressor with optimized hyperparameters
gbr_best = GradientBoostingRegressor(
    max_depth=gs.best_params_['max_depth'],
    n_estimators=gs.best_params_['n_estimators'],
    max_features=gs.best_params_['max_features'])

with joblib.paralell_backend('dask'):
  gbr_best.fit(X_train, Y_train)

print(gbr_best.score(X_train, Y_train))
print(gbr_best.score(X_test, Y_test))

In [0]:
pd.Series(gbr.feature_importances_, index=X_train.columns).sort_values(ascending=False)

## Evaluate
### Print metrics for the test

In [0]:
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.metrics import mean_absolute_error

preds = gbr_best.predict(X_test)

ds.print_evaluation_metrics(Y_test, preds)

## 5. Results
While the model could use some improvement, given the time constraint on this project, a score of .84 was a satisfactory enough R^2 to support the idea that yes, you can predict solar generation at an hour given other generations and weather. 

## 6. Discussion and Recommendation
Solar is still a pretty hot topic for homeowners and energy production companies around the globe. While it is important to remember that without seemingly impossible storage systems and planning, it is not a viable baseload solution. Knowing how much production to expect at any time of the day is vital to keeping the energy demand met and preventing transmission overloads.

If I were to take this project further, I would use the time-series attributes of my data to create a forecast model that could predict future generation instead of current generation.